# Datathon — Avaliação do Modelo

Pipeline de avaliação do modelo de alerta preventivo de risco educacional. Consome o modelo treinado pelo `train.ipynb` e os datasets de features para produzir métricas operacionais, relatórios por fase e análises de interpretabilidade.

---

## Visão Geral

| Etapa | Descrição |
|---|---|
| **1. Configuração** | Caminhos, constantes e parâmetros de orçamento operacional (`K_LIST`) |
| **2. Avaliação Operacional** | Recall@TopK, Precision@TopK e Lift@TopK com política estratificada por fase |
| **3. Feature Importance** | Importância global via `LossFunctionChange` por fase |
| **4. Análise SHAP** | SHAP médio por fase com direção do efeito (↑ risco / ↓ risco) |
| **5. Exportação** | Métricas, lista de alertas e SHAP salvos em `notebooks/evaluation/` |

---

## Métrica Principal

$$\text{Recall@TopK} = \frac{\text{alunos em risco real dentro do Top-K\%}}{\text{total de alunos em risco real}}$$

A política de alertas é **estratificada por fase**: o Top-K% é calculado separadamente dentro de cada fase, garantindo representatividade independente do tamanho do grupo.

---

## Saídas deste Notebook

| Arquivo | Conteúdo |
|---|---|
| `metrics__piora__*.json` | AUC global + Recall/Precision/Lift@K para cada valor de K |
| `topk_by_fase__k15__*.parquet` | Métricas detalhadas por fase para K=15% |
| `alerts__k15__*.parquet` | Lista final de alunos em alerta (Top-15% por fase) |
| `valid_out__*.parquet` | Dataset de validação completo com `proba` e `score` |
| `shap_by_fase__long.parquet` | SHAP por fase em formato longo |
| `shap_by_fase__top_features.parquet / .csv` | Top features por fase com direção do efeito |


In [29]:
# catboost já instalado no ambiente local (.venv)
# Para instalar manualmente: pip install catboost
import catboost
print("✅ catboost version:", catboost.__version__)


✅ catboost version: 1.2.10


## 2. Avaliação Operacional — Recall@TopK Estratificado por Fase

Executa o pipeline completo de avaliação: carrega dados e modelo, gera scores, aplica a política Top-K% por fase e calcula métricas para K ∈ {10, 15, 20, 25}. Os artefatos são salvos automaticamente em `notebooks/evaluation/`.


## 1. Configuração e Carregamento

Importa dependências, define caminhos, constantes operacionais e carrega os artefatos produzidos pelo pipeline de treino.

| Constante | Valor padrão | Significado |
|---|---|---|
| `K_MAIN` | 15 | K principal usado nos relatórios e na lista de alertas |
| `K_LIST` | `[10, 15, 20, 25]` | Grade de K avaliados — permite o coordenador escolher o orçamento |
| `FEAT_DIR` | `notebooks/` | Origem dos Parquets de features |
| `OUT_DIR` | `notebooks/evaluation/` | Destino de todas as saídas |


In [30]:
# evaluate.py — execução local

import json
import numpy as np
import pandas as pd
from pathlib import Path

# ----------------------------
# 0) CONFIGURAÇÃO DE CAMINHOS (local)
# ----------------------------
ROOT_DIR = Path("..").resolve()          # notebooks/ -> workspace root
NOTEBOOKS_DIR = ROOT_DIR / "notebooks"

FEAT_DIR = NOTEBOOKS_DIR / "data"
ART_DIR  = NOTEBOOKS_DIR / "models"
OUT_DIR  = NOTEBOOKS_DIR / "evaluation"

TRAIN_FEAT_FILE   = "train_feat__piora__2022_2023.parquet"
VALID_FEAT_FILE   = "valid_feat__piora__2023_2024.parquet"
MODEL_BUNDLE_FILE = "catboost__piora__2022_2023.joblib"

# K principal do "orçamento operacional"
K_MAIN = 15
K_LIST = [10, 15, 20, 25]

TARGET_COL = "target"
FASE_COL   = "fase"
SCORE_COL  = "score"
PROBA_COL  = "proba"
ALERT_COL  = "alerta"

OUT_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) FUNÇÕES TOP-K E MÉTRICAS
# ----------------------------
def stratified_topk_alert(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> pd.DataFrame:
    out = df.copy()
    out[alert_col] = 0
    for fase, g in out.groupby(fase_col, dropna=False):
        n = len(g)
        if n == 0:
            continue
        k = max(1, int(np.ceil(n * k_pct / 100)))
        idx = g.sort_values(score_col, ascending=False).head(k).index
        out.loc[idx, alert_col] = 1
    return out


def operational_metrics_topk(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    target_col: str = TARGET_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> dict:
    tmp = stratified_topk_alert(df, score_col, fase_col, k_pct, alert_col)
    y = tmp[target_col].astype(int).values
    a = tmp[alert_col].astype(int).values
    tp = int(((a == 1) & (y == 1)).sum())
    fp = int(((a == 1) & (y == 0)).sum())
    fn = int(((a == 0) & (y == 1)).sum())
    recall    = tp / (tp + fn) if (tp + fn) else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    base_rate = float(y.mean()) if len(y) else 0.0
    lift      = (precision / base_rate) if base_rate > 0 else np.nan
    return {
        "k_pct": float(k_pct), "n_total": int(len(y)), "n_alert": int(a.sum()),
        "n_pos": int(y.sum()), "base_rate": base_rate, "tp": tp, "fp": fp, "fn": fn,
        "recall@k": float(recall), "precision@k": float(precision),
        "lift@k": float(lift) if np.isfinite(lift) else None,
        "df_with_alerts": tmp,
    }


def operational_metrics_topk_by_fase(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    target_col: str = TARGET_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> pd.DataFrame:
    tmp = stratified_topk_alert(df, score_col, fase_col, k_pct, alert_col)
    rows = []
    for fase, g in tmp.groupby(fase_col, dropna=False):
        y = g[target_col].astype(int).values
        a = g[alert_col].astype(int).values
        tp = int(((a == 1) & (y == 1)).sum())
        fp = int(((a == 1) & (y == 0)).sum())
        fn = int(((a == 0) & (y == 1)).sum())
        recall    = tp / (tp + fn) if (tp + fn) else 0.0
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        base_rate = float(y.mean()) if len(y) else 0.0
        lift      = (precision / base_rate) if base_rate > 0 else np.nan
        rows.append({
            fase_col: fase, "n": int(len(g)), "n_alert": int(a.sum()),
            "n_pos": int(y.sum()), "base_rate": base_rate,
            "recall@k": float(recall), "precision@k": float(precision),
            "lift@k": float(lift) if np.isfinite(lift) else None,
        })
    return pd.DataFrame(rows).sort_values(by=fase_col).reset_index(drop=True)


# ----------------------------
# 2) CARREGAR DADOS (PARQUET)
# ----------------------------
train_feat = pd.read_parquet(FEAT_DIR / TRAIN_FEAT_FILE)
valid_feat  = pd.read_parquet(FEAT_DIR / VALID_FEAT_FILE)

print("✅ Dados carregados de:", FEAT_DIR)
print("  train_feat:", train_feat.shape, "  valid_feat:", valid_feat.shape)

# ----------------------------
# 3) CARREGAR MODELO (JOBLIB)
# ----------------------------
import joblib
from catboost import Pool
from sklearn.metrics import roc_auc_score

bundle = joblib.load(ART_DIR / MODEL_BUNDLE_FILE)

model        = bundle["model"]
feature_cols = bundle["feature_cols"]
cat_features = bundle["cat_features"]
target_col   = bundle.get("target", TARGET_COL)

print("✅ Modelo carregado de:", ART_DIR / MODEL_BUNDLE_FILE)

# ----------------------------
# 4) GERAR SCORES NA VALIDAÇÃO
# ----------------------------
X_valid = valid_feat[feature_cols].copy()
y_valid = valid_feat[target_col].astype(int).copy()

for c in cat_features:
    if c in X_valid.columns:
        X_valid[c] = X_valid[c].astype(str)

cat_idx     = [X_valid.columns.get_loc(c) for c in cat_features if c in X_valid.columns]
valid_pool  = Pool(X_valid, y_valid, cat_features=cat_idx)

valid_proba = model.predict_proba(valid_pool)[:, 1]
valid_score = (100 * valid_proba).round(4)

auc = roc_auc_score(y_valid, valid_proba)

valid_out            = valid_feat.copy()
valid_out[PROBA_COL] = valid_proba
valid_out[SCORE_COL] = valid_score

# ----------------------------
# 5) MÉTRICAS TOP-K (10/15/20/25)
# ----------------------------
metrics = {
    "auc_valid":      float(auc),
    "n_valid":        int(len(valid_out)),
    "base_rate_valid": float(valid_out[target_col].mean()),
    "topk":           [],
    "k_main":         K_MAIN,
}

print(f"\n✅ AUC (valid): {auc:.6f}")
print(f"✅ Base rate (valid): {metrics['base_rate_valid']:.4f} | N={metrics['n_valid']}")

for k in K_LIST:
    res = operational_metrics_topk(
        valid_out, score_col=SCORE_COL, target_col=target_col,
        fase_col=FASE_COL, k_pct=k, alert_col=ALERT_COL,
    )
    metrics["topk"].append({kk: vv for kk, vv in res.items() if kk != "df_with_alerts"})
    print(
        f"K={k}% | Recall@K={res['recall@k']:.3f} | "
        f"Precision@K={res['precision@k']:.3f} | "
        f"Lift@K={res['lift@k'] if res['lift@k'] is not None else np.nan:.2f} | "
        f"Alerts={res['n_alert']}/{res['n_total']}"
    )

# ----------------------------
# 6) RELATÓRIO POR FASE (K=15)
# ----------------------------
topk_by_fase = operational_metrics_topk_by_fase(
    valid_out, score_col=SCORE_COL, target_col=target_col,
    fase_col=FASE_COL, k_pct=K_MAIN, alert_col=ALERT_COL,
)

# ----------------------------
# 7) LISTA DE ALERTAS (K=15)
# ----------------------------
res_main  = operational_metrics_topk(
    valid_out, score_col=SCORE_COL, target_col=target_col,
    fase_col=FASE_COL, k_pct=K_MAIN, alert_col=ALERT_COL,
)
alerts_df = res_main["df_with_alerts"].copy()
alerts_df = alerts_df.sort_values([FASE_COL, SCORE_COL], ascending=[True, False])
alerts_only = alerts_df[alerts_df[ALERT_COL] == 1].copy()

# ----------------------------
# 8) SALVAR ARTEFATOS (locais)
# ----------------------------
stem          = "piora__train_2022_2023__valid_2023_2024"
metrics_path  = OUT_DIR / f"metrics__{stem}.json"
byfase_path   = OUT_DIR / f"topk_by_fase__k{K_MAIN}__{stem}.parquet"
alerts_path   = OUT_DIR / f"alerts__k{K_MAIN}__{stem}.parquet"
validout_path = OUT_DIR / f"valid_out__{stem}.parquet"

with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

topk_by_fase.to_parquet(byfase_path, index=False)
alerts_only.to_parquet(alerts_path, index=False)
valid_out.to_parquet(validout_path, index=False)

print("\n✅ Arquivos salvos em:", OUT_DIR)
print(" -", metrics_path.name)
print(" -", byfase_path.name)
print(" -", alerts_path.name)
print(" -", validout_path.name)

display(topk_by_fase.head(20))
display(alerts_only[[c for c in ["ano_base","fase","turma","score","proba","alerta",target_col] if c in alerts_only.columns]].head(30))


✅ Dados carregados de: /home/glauberthy/Desktop/datathon/notebooks/data
  train_feat: (600, 122)   valid_feat: (765, 122)
✅ Modelo carregado de: /home/glauberthy/Desktop/datathon/notebooks/models/catboost__piora__2022_2023.joblib

✅ AUC (valid): 0.715146
✅ Base rate (valid): 0.4092 | N=765
K=10% | Recall@K=0.192 | Precision@K=0.750 | Lift@K=1.83 | Alerts=80/765
K=15% | Recall@K=0.275 | Precision@K=0.723 | Lift@K=1.77 | Alerts=119/765
K=20% | Recall@K=0.342 | Precision@K=0.686 | Lift@K=1.68 | Alerts=156/765
K=25% | Recall@K=0.409 | Precision@K=0.656 | Lift@K=1.60 | Alerts=195/765

✅ Arquivos salvos em: /home/glauberthy/Desktop/datathon/notebooks/evaluation
 - metrics__piora__train_2022_2023__valid_2023_2024.json
 - topk_by_fase__k15__piora__train_2022_2023__valid_2023_2024.parquet
 - alerts__k15__piora__train_2022_2023__valid_2023_2024.parquet
 - valid_out__piora__train_2022_2023__valid_2023_2024.parquet


,fase,n,n_alert,n_pos,base_rate,recall@k,precision@k,lift@k
0,0,174,27,40,0.229885,0.550000,0.814815,3.544444
1,1,138,21,72,0.521739,0.250000,0.857143,1.642857
2,2,153,23,101,0.660131,0.188119,0.826087,1.251399
3,3,94,15,34,0.361702,0.294118,0.666667,1.843137
4,4,67,11,24,0.358209,0.208333,0.454545,1.268939
5,5,43,7,14,0.325581,0.285714,0.571429,1.755102
6,6,17,3,10,0.588235,0.300000,1.000000,1.700000
7,7,20,3,5,0.250000,0.600000,1.000000,4.000000
8,8,59,9,13,0.220339,0.153846,0.222222,1.008547


,ano_base,fase,turma,score,proba,alerta,target
587,2023,0,ALFA O - G2/G3,51.1040,0.511040,1,1
590,2023,0,ALFA L - G2/G3,49.5107,0.495107,1,1
570,2023,0,ALFA E - G2/G3,48.3895,0.483895,1,1
600,2023,0,ALFA N - G2/G3,48.1654,0.481654,1,1
729,2023,0,ALFA N - G2/G3,47.7721,0.477721,1,1
591,2023,0,ALFA N - G2/G3,47.4021,0.474021,1,1
554,2023,0,ALFA I - G2/G3,47.0137,0.470137,1,1
547,2023,0,ALFA G - G2/G3,46.8899,0.468899,1,1
764,2023,0,ALFA T - G2/G3,46.5913,0.465913,1,0
582,2023,0,ALFA O - G2/G3,46.4290,0.464290,1,1


### Funções de Avaliação Operacional

As três funções abaixo implementam a política Top-K% estratificada por fase:

| Função | O que faz |
|---|---|
| `stratified_topk_alert` | Marca os Top-K% alunos por score **dentro de cada fase** com `alerta=1` |
| `operational_metrics_topk` | Calcula Recall@K, Precision@K e Lift@K globais usando a política estratificada |
| `operational_metrics_topk_by_fase` | Idem, mas desagregado por fase — útil para detectar fases com baixa cobertura |

> **Por que estratificar por fase?** Sem estratificação, fases maiores dominam a lista de alertas e fases menores (ex.: ALFA) ficam sub-representadas — exatamente o oposto do desejado operacionalmente.


In [31]:
# ----------------------------
# 1) FUNÇÕES TOP-K E MÉTRICAS
# ----------------------------
def stratified_topk_alert(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> pd.DataFrame:
    """
    Marca Top-K% por score dentro de cada fase (estratificado).
    """
    out = df.copy()
    out[alert_col] = 0

    for fase, g in out.groupby(fase_col, dropna=False):
        n = len(g)
        if n == 0:
            continue
        k = max(1, int(np.ceil(n * k_pct / 100)))
        idx = g.sort_values(score_col, ascending=False).head(k).index
        out.loc[idx, alert_col] = 1

    return out


def operational_metrics_topk(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    target_col: str = TARGET_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> dict:
    """
    Métricas globais com política estratificada por fase.
    Retorna também df_with_alerts (para gerar lista final).
    """
    tmp = stratified_topk_alert(df, score_col, fase_col, k_pct, alert_col)

    y = tmp[target_col].astype(int).values
    a = tmp[alert_col].astype(int).values

    tp = int(((a == 1) & (y == 1)).sum())
    fp = int(((a == 1) & (y == 0)).sum())
    fn = int(((a == 0) & (y == 1)).sum())

    recall = tp / (tp + fn) if (tp + fn) else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    base_rate = float(y.mean()) if len(y) else 0.0
    lift = (precision / base_rate) if base_rate > 0 else np.nan

    return {
        "k_pct": float(k_pct),
        "n_total": int(len(y)),
        "n_alert": int(a.sum()),
        "n_pos": int(y.sum()),
        "base_rate": base_rate,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "recall@k": float(recall),
        "precision@k": float(precision),
        "lift@k": float(lift) if np.isfinite(lift) else None,
        "df_with_alerts": tmp,
    }


def operational_metrics_topk_by_fase(
    df: pd.DataFrame,
    score_col: str = SCORE_COL,
    target_col: str = TARGET_COL,
    fase_col: str = FASE_COL,
    k_pct: float = 15.0,
    alert_col: str = ALERT_COL,
) -> pd.DataFrame:
    """
    Métricas por fase (útil para diagnóstico e explicação operacional).
    """
    tmp = stratified_topk_alert(df, score_col, fase_col, k_pct, alert_col)

    rows = []
    for fase, g in tmp.groupby(fase_col, dropna=False):
        y = g[target_col].astype(int).values
        a = g[alert_col].astype(int).values

        tp = int(((a == 1) & (y == 1)).sum())
        fp = int(((a == 1) & (y == 0)).sum())
        fn = int(((a == 0) & (y == 1)).sum())

        recall = tp / (tp + fn) if (tp + fn) else 0.0
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        base_rate = float(y.mean()) if len(y) else 0.0
        lift = (precision / base_rate) if base_rate > 0 else np.nan

        rows.append({
            fase_col: fase,
            "n": int(len(g)),
            "n_alert": int(a.sum()),
            "n_pos": int(y.sum()),
            "base_rate": base_rate,
            "recall@k": float(recall),
            "precision@k": float(precision),
            "lift@k": float(lift) if np.isfinite(lift) else None,
        })

    return pd.DataFrame(rows).sort_values(by=fase_col).reset_index(drop=True)


# ----------------------------
# 2) CARREGAR DADOS (PARQUET)
# ----------------------------
train_feat = pd.read_parquet(FEAT_DIR / TRAIN_FEAT_FILE)
valid_feat = pd.read_parquet(FEAT_DIR / VALID_FEAT_FILE)

# ----------------------------
# 3) CARREGAR MODELO (JOBLIB)
# ----------------------------
import joblib
from catboost import Pool
from sklearn.metrics import roc_auc_score

bundle = joblib.load(ART_DIR / MODEL_BUNDLE_FILE)

model = bundle["model"]
feature_cols = bundle["feature_cols"]
cat_features = bundle["cat_features"]
target_col = bundle.get("target", TARGET_COL)

# ----------------------------
# 4) GERAR SCORES NA VALIDAÇÃO
# ----------------------------
X_valid = valid_feat[feature_cols].copy()
y_valid = valid_feat[target_col].astype(int).copy()

# CatBoost: categóricas como string
for c in cat_features:
    if c in X_valid.columns:
        X_valid[c] = X_valid[c].astype(str)

cat_idx = [X_valid.columns.get_loc(c) for c in cat_features if c in X_valid.columns]
valid_pool = Pool(X_valid, y_valid, cat_features=cat_idx)

valid_proba = model.predict_proba(valid_pool)[:, 1]
valid_score = (100 * valid_proba).round(4)

auc = roc_auc_score(y_valid, valid_proba)

valid_out = valid_feat.copy()
valid_out[PROBA_COL] = valid_proba
valid_out[SCORE_COL] = valid_score

# ----------------------------
# 5) MÉTRICAS TOP-K (10/15/20/25)
# ----------------------------
metrics = {
    "auc_valid": float(auc),
    "n_valid": int(len(valid_out)),
    "base_rate_valid": float(valid_out[target_col].mean()),
    "topk": [],
    "k_main": K_MAIN,
}

print(f"✅ AUC (valid): {auc:.6f}")
print(f"✅ Base rate (valid): {metrics['base_rate_valid']:.4f} | N={metrics['n_valid']}")

for k in K_LIST:
    res = operational_metrics_topk(
        valid_out,
        score_col=SCORE_COL,
        target_col=target_col,
        fase_col=FASE_COL,
        k_pct=k,
        alert_col=ALERT_COL,
    )
    metrics["topk"].append({kk: vv for kk, vv in res.items() if kk != "df_with_alerts"})

    print(
        f"K={k}% | Recall@K={res['recall@k']:.3f} | "
        f"Precision@K={res['precision@k']:.3f} | "
        f"Lift@K={res['lift@k'] if res['lift@k'] is not None else np.nan:.2f} | "
        f"Alerts={res['n_alert']}/{res['n_total']}"
    )

# ----------------------------
# 6) RELATÓRIO POR FASE (K=15)
# ----------------------------
topk_by_fase = operational_metrics_topk_by_fase(
    valid_out,
    score_col=SCORE_COL,
    target_col=target_col,
    fase_col=FASE_COL,
    k_pct=K_MAIN,
    alert_col=ALERT_COL,
)

# ----------------------------
# 7) LISTA DE ALERTAS (K=15)
# ----------------------------
res_main = operational_metrics_topk(
    valid_out,
    score_col=SCORE_COL,
    target_col=target_col,
    fase_col=FASE_COL,
    k_pct=K_MAIN,
    alert_col=ALERT_COL,
)
alerts_df = res_main["df_with_alerts"].copy()
alerts_df = alerts_df.sort_values([FASE_COL, SCORE_COL], ascending=[True, False])

alerts_only = alerts_df[alerts_df[ALERT_COL] == 1].copy()

# ----------------------------
# 8) SALVAR ARTEFATOS
# ----------------------------
# nomes dos arquivos
stem = "piora__train_2022_2023__valid_2023_2024"
metrics_path = OUT_DIR / f"metrics__{stem}.json"
byfase_path = OUT_DIR / f"topk_by_fase__k{K_MAIN}__{stem}.parquet"
alerts_path = OUT_DIR / f"alerts__k{K_MAIN}__{stem}.parquet"
validout_path = OUT_DIR / f"valid_out__{stem}.parquet"

with open(metrics_path, "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

topk_by_fase.to_parquet(byfase_path, index=False)
alerts_only.to_parquet(alerts_path, index=False)
valid_out.to_parquet(validout_path, index=False)

print("\n✅ Arquivos salvos em:", OUT_DIR)
print(" -", metrics_path.name)
print(" -", byfase_path.name)
print(" -", alerts_path.name)
print(" -", validout_path.name)

# (Opcional) exibir amostras no notebook
display(topk_by_fase.head(20))
display(alerts_only[[c for c in ["ano_base","fase","turma","score","proba","alerta",target_col] if c in alerts_only.columns]].head(30))

✅ AUC (valid): 0.715146
✅ Base rate (valid): 0.4092 | N=765
K=10% | Recall@K=0.192 | Precision@K=0.750 | Lift@K=1.83 | Alerts=80/765
K=15% | Recall@K=0.275 | Precision@K=0.723 | Lift@K=1.77 | Alerts=119/765
K=20% | Recall@K=0.342 | Precision@K=0.686 | Lift@K=1.68 | Alerts=156/765
K=25% | Recall@K=0.409 | Precision@K=0.656 | Lift@K=1.60 | Alerts=195/765

✅ Arquivos salvos em: /home/glauberthy/Desktop/datathon/notebooks/evaluation
 - metrics__piora__train_2022_2023__valid_2023_2024.json
 - topk_by_fase__k15__piora__train_2022_2023__valid_2023_2024.parquet
 - alerts__k15__piora__train_2022_2023__valid_2023_2024.parquet
 - valid_out__piora__train_2022_2023__valid_2023_2024.parquet


,fase,n,n_alert,n_pos,base_rate,recall@k,precision@k,lift@k
0,0,174,27,40,0.229885,0.550000,0.814815,3.544444
1,1,138,21,72,0.521739,0.250000,0.857143,1.642857
2,2,153,23,101,0.660131,0.188119,0.826087,1.251399
3,3,94,15,34,0.361702,0.294118,0.666667,1.843137
4,4,67,11,24,0.358209,0.208333,0.454545,1.268939
5,5,43,7,14,0.325581,0.285714,0.571429,1.755102
6,6,17,3,10,0.588235,0.300000,1.000000,1.700000
7,7,20,3,5,0.250000,0.600000,1.000000,4.000000
8,8,59,9,13,0.220339,0.153846,0.222222,1.008547


,ano_base,fase,turma,score,proba,alerta,target
587,2023,0,ALFA O - G2/G3,51.1040,0.511040,1,1
590,2023,0,ALFA L - G2/G3,49.5107,0.495107,1,1
570,2023,0,ALFA E - G2/G3,48.3895,0.483895,1,1
600,2023,0,ALFA N - G2/G3,48.1654,0.481654,1,1
729,2023,0,ALFA N - G2/G3,47.7721,0.477721,1,1
591,2023,0,ALFA N - G2/G3,47.4021,0.474021,1,1
554,2023,0,ALFA I - G2/G3,47.0137,0.470137,1,1
547,2023,0,ALFA G - G2/G3,46.8899,0.468899,1,1
764,2023,0,ALFA T - G2/G3,46.5913,0.465913,1,0
582,2023,0,ALFA O - G2/G3,46.4290,0.464290,1,1


## 3. Interpretabilidade — Feature Importance

In [32]:
X_train = train_feat[feature_cols].copy()

importances = model.get_feature_importance()
feat_imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importances
}).sort_values("importance", ascending=False)

feat_imp.head(15)

,feature,importance
6,ano_nasc,9.217033
8,defasagem_t,8.907608
9,defasagem_t_is0,3.479200
17,disp_provas,2.216685
1,turma,2.010534
0,fase,1.996532
99,delta_fase_matem,1.689904
2,genero,1.638834
105,delta_turma_ieg,1.611793
106,z_turma_ieg,1.583489


### 3.1 Importância por Fase — `LossFunctionChange`

Calcula a importância de cada feature separadamente para cada fase, medindo quanto cada variável contribui para reduzir a `Logloss` naquele grupo específico.

| Métrica | Interpretação |
|---|---|
| `importance` alta | A feature é muito relevante para a predição nesta fase |
| `unstable=True` | Fase com menos de `min_n=30` alunos — importância menos confiável |

> `LossFunctionChange` é preferível ao `PredictionValuesChange` quando há desbalanceamento de classes, pois mede impacto direto na função de perda e não apenas na variação da predição.


In [33]:
import numpy as np
import pandas as pd
from catboost import Pool

def feature_importance_by_fase(
    model,
    df: pd.DataFrame,
    feature_cols: list[str],
    cat_features: list[str],
    target_col: str = "target",
    fase_col: str = "fase",
    importance_type: str = "LossFunctionChange",  # ou "PredictionValuesChange"
    min_n: int = 30,
    top_n: int = 20,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Retorna:
      - long_df: importance por fase em formato longo (fase, feature, importance)
      - top_df : top-N por fase (já ordenado)
    """

    long_rows = []
    fases = sorted(df[fase_col].dropna().unique().tolist())

    for fase in fases:
        g = df[df[fase_col] == fase].copy()
        n = len(g)
        if n < min_n:
            # ainda calcula, mas marca como instável
            pass

        X = g[feature_cols].copy()
        y = g[target_col].astype(int).copy()

        # garante tipagem das categóricas como string (CatBoost gosta)
        for c in cat_features:
            if c in X.columns:
                X[c] = X[c].astype(str)

        cat_idx = [X.columns.get_loc(c) for c in cat_features if c in X.columns]
        pool = Pool(X, y, cat_features=cat_idx)

        imps = model.get_feature_importance(pool, type=importance_type)
        # imps vem alinhado à ordem de feature_cols
        for feat, imp in zip(feature_cols, imps):
            long_rows.append({
                "fase": fase,
                "n": n,
                "feature": feat,
                "importance": float(imp),
                "importance_type": importance_type,
                "unstable": bool(n < min_n),
            })

    long_df = pd.DataFrame(long_rows)

    # top-N por fase
    top_df = (
        long_df.sort_values(["fase", "importance"], ascending=[True, False])
              .groupby("fase", as_index=False)
              .head(top_n)
              .reset_index(drop=True)
    )

    return long_df, top_df


# ======= USO =======
# (A) se você já está no evaluate e tem:
# model, valid_out (ou valid_feat), feature_cols, cat_features
# e target_col, fase_col

# escolha a base: recomendo valid_out (tem o mesmo target e as features)
df_base = valid_out  # ou valid_feat, desde que tenha target e features

long_imp, top_imp = feature_importance_by_fase(
    model=model,
    df=df_base,
    feature_cols=feature_cols,
    cat_features=cat_features,
    target_col="target",
    fase_col="fase",
    importance_type="LossFunctionChange",
    min_n=30,
    top_n=20,
)

display(top_imp.head(40))



,fase,n,feature,importance,importance_type,unstable
0,0,174,defasagem_t,0.152994,LossFunctionChange,False
1,0,174,defasagem_t_is0,0.037180,LossFunctionChange,False
2,0,174,ano_nasc,0.015314,LossFunctionChange,False
3,0,174,turma_mean_iaa,0.007894,LossFunctionChange,False
4,0,174,fase_std_ips,0.005150,LossFunctionChange,False
5,0,174,fase__p25_ieg,0.004991,LossFunctionChange,False
6,0,174,turma__p75_media_provas,0.004218,LossFunctionChange,False
7,0,174,turma_mean_matem,0.004152,LossFunctionChange,False
8,0,174,z_turma_iaa,0.003789,LossFunctionChange,False
9,0,174,turma__p25_media_provas,0.003672,LossFunctionChange,False


## 4. Interpretabilidade — Análise SHAP por Fase

SHAP (*SHapley Additive exPlanations*) decompõe a predição de cada aluno na contribuição individual de cada feature, permitindo entender **qual variável aumenta ou reduz o risco** dentro de cada fase.

| Abordagem | Vantagem | Custo |
|---|---|---|
| `LossFunctionChange` (seção anterior) | Rápido, mede importância relativa | Não tem direção do efeito |
| **SHAP** (esta seção) | Tem direção (↑ risco / ↓ risco) e magnitude por aluno | Mais lento — usa `sample_per_fase=500` por padrão |

> **Quando usar SHAP?** Para comunicar aos coordenadores *por que* o modelo gerou um alerta — ex.: "aluno X: media_provas baixa (+0.12 risco) e tempo_casa alto (−0.04 risco)".


### 4.1 Função `shap_mean_by_fase`

Calcula o SHAP médio por fase, agregando as contribuições individuais de cada aluno:

| Campo de saída | Significado |
|---|---|
| `mean_shap` | Contribuição média com sinal — positivo = aumenta risco, negativo = reduz risco |
| `mean_abs_shap` | Força média (magnitude) — usada para ranquear as features mais influentes |
| `pct_positive` | % dos alunos na fase em que a feature aumentou o risco |

O parâmetro `sample_per_fase` limita o processamento a 500 alunos por fase por padrão — aumente para maior precisão ou use `None` para processar todos.


In [34]:
import numpy as np
import pandas as pd
from catboost import Pool

def shap_mean_by_fase(
    model,
    df: pd.DataFrame,
    feature_cols: list[str],
    cat_features: list[str],
    target_col: str = "target",
    fase_col: str = "fase",
    sample_per_fase: int | None = 500,   # None = usa tudo (mais lento)
    random_state: int = 42,
    top_n: int = 20,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Calcula SHAP médio por fase (com direção do efeito).

    Retorna:
      - long_df: (fase, n, feature, mean_shap, mean_abs_shap, pct_positive)
      - top_df : top-N por fase (ordenado por mean_abs_shap)
    """
    rng = np.random.default_rng(random_state)
    rows = []

    fases = sorted(df[fase_col].dropna().unique().tolist())

    for fase in fases:
        g = df[df[fase_col] == fase].copy()
        n0 = len(g)
        if n0 == 0:
            continue

        # amostragem para ficar viável
        if sample_per_fase is not None and n0 > sample_per_fase:
            idx = rng.choice(g.index.to_numpy(), size=sample_per_fase, replace=False)
            g = g.loc[idx].copy()

        n = len(g)

        X = g[feature_cols].copy()

        # CatBoost gosta de categóricas como string
        for c in cat_features:
            if c in X.columns:
                X[c] = X[c].astype(str)

        cat_idx = [X.columns.get_loc(c) for c in cat_features if c in X.columns]
        # y não é necessário para SHAP, mas pode ser incluído sem problema
        y = g[target_col].astype(int).values if target_col in g.columns else None
        pool = Pool(X, label=y, cat_features=cat_idx)

        # SHAP: retorna matriz (n, m+1); última coluna é o "base value"
        shap_matrix = model.get_feature_importance(pool, type="ShapValues")
        shap_vals = shap_matrix[:, :-1]  # (n, m) contribuições por feature

        mean_shap = shap_vals.mean(axis=0)                 # direção média
        mean_abs = np.abs(shap_vals).mean(axis=0)          # força média
        pct_pos  = (shap_vals > 0).mean(axis=0)            # % vezes que aumenta risco

        for feat, ms, ma, pp in zip(feature_cols, mean_shap, mean_abs, pct_pos):
            rows.append({
                "fase": fase,
                "n": n,
                "feature": feat,
                "mean_shap": float(ms),
                "mean_abs_shap": float(ma),
                "pct_positive": float(pp),
            })

    long_df = pd.DataFrame(rows)

    # Top-N por fase: ordena pela força (mean_abs_shap)
    top_df = (
        long_df.sort_values(["fase", "mean_abs_shap"], ascending=[True, False])
              .groupby("fase", as_index=False)
              .head(top_n)
              .reset_index(drop=True)
    )

    return long_df, top_df

In [35]:
# Recomendo usar valid_out (ou valid_feat); SHAP não precisa de score/proba
df_base = valid_out  # ou valid_feat

long_shap, top_shap = shap_mean_by_fase(
    model=model,
    df=df_base,
    feature_cols=feature_cols,
    cat_features=cat_features,
    target_col="target",
    fase_col="fase",
    sample_per_fase=500,  # aumente para 1000 ou None se quiser mais precisão
    top_n=15
)

display(top_shap.head(50))

,fase,n,feature,mean_shap,mean_abs_shap,pct_positive
0,0,174,defasagem_t,-0.077414,0.366358,0.534483
1,0,174,defasagem_t_is0,-0.050691,0.169664,0.534483
2,0,174,turma_mean_iaa,0.083437,0.083437,1.000000
3,0,174,genero,0.001698,0.081319,0.517241
4,0,174,fase__p25_ieg,0.066488,0.066488,1.000000
5,0,174,delta_turma_iaa,-0.064149,0.064149,0.000000
6,0,174,delta_fase_media_provas,-0.063407,0.063407,0.000000
7,0,174,delta_fase_ips,0.059631,0.059631,1.000000
8,0,174,turma__p75_ieg,-0.057685,0.057685,0.000000
9,0,174,turma__p75_media_provas,0.055722,0.055722,1.000000


In [36]:
rep = top_shap.copy()
rep["direction"] = np.where(rep["mean_shap"] >= 0, "↑ risco", "↓ risco")
rep["mean_shap"] = rep["mean_shap"].round(6)
rep["mean_abs_shap"] = rep["mean_abs_shap"].round(6)
rep["pct_positive"] = (100 * rep["pct_positive"]).round(1)

# Reorganiza colunas
rep = rep[["fase", "n", "feature", "mean_abs_shap", "mean_shap", "direction", "pct_positive"]]
rep.rename(columns={"pct_positive": "% vezes ↑ risco"}, inplace=True)

display(rep)

,fase,n,feature,mean_abs_shap,mean_shap,direction,% vezes ↑ risco
0,0,174,defasagem_t,0.366358,-0.077414,↓ risco,53.4
1,0,174,defasagem_t_is0,0.169664,-0.050691,↓ risco,53.4
2,0,174,turma_mean_iaa,0.083437,0.083437,↑ risco,100.0
3,0,174,genero,0.081319,0.001698,↑ risco,51.7
4,0,174,fase__p25_ieg,0.066488,0.066488,↑ risco,100.0
...,...,...,...,...,...,...,...
130,8,59,ieg,0.061572,-0.061572,↓ risco,0.0
131,8,59,delta_fase_media_provas,0.059263,-0.059263,↓ risco,0.0
132,8,59,delta_turma_media_provas,0.057366,-0.057366,↓ risco,0.0
133,8,59,delta_fase_ieg,0.052806,-0.052806,↓ risco,0.0


### 4.2 Exportação dos Resultados SHAP

Persiste os resultados SHAP em três formatos:

| Arquivo | Formato | Uso |
|---|---|---|
| `shap_by_fase__long.parquet` | Parquet | Análises downstream — todas as features e fases |
| `shap_by_fase__top_features.parquet` | Parquet | Top features por fase com direção |
| `shap_by_fase__top_features.csv` | CSV | Relatório para coordenadores / cola em Excel |


In [37]:
SHAP_DIR = OUT_DIR  # já definido como NOTEBOOKS_DIR / "evaluation" na célula de config
SHAP_DIR.mkdir(parents=True, exist_ok=True)

long_shap.to_parquet(SHAP_DIR / "shap_by_fase__long.parquet", index=False)
rep.to_parquet(SHAP_DIR / "shap_by_fase__top_features.parquet", index=False)

# Também útil em CSV para colar em relatório/Excel
rep.to_csv(SHAP_DIR / "shap_by_fase__top_features.csv", index=False, encoding="utf-8")

print("✅ SHAP por fase salvo em:", SHAP_DIR)


✅ SHAP por fase salvo em: /home/glauberthy/Desktop/datathon/notebooks/evaluation
